# Limpieza de Datos

In [1]:
import pandas as pd
import unicodedata
import re

## Limpieza de la variable "Teléfono"

In [313]:
df = pd.read_csv("all_data.csv", dtype={"TELEFONO": str})

In [314]:
# Cuántos valores únicos y cuántos vacíos
print(df["TELEFONO"].nunique(), "valores únicos")
print(df["TELEFONO"].isna().sum(), "NaN / vacíos")
print("Total rows:", df.shape[0])

4215 valores únicos
46 NaN / vacíos
Total rows: 6599


In [315]:
# Valores con notación científica (“.0”)  
mask_sci = df["TELEFONO"].str.contains(r"\.0$", na=False)
print("Con .0 al final:", mask_sci.sum())

# Valores con caracteres no numéricos  
mask_nonnumeric = df["TELEFONO"].str.contains(r"\D", na=False)
print("Con otros símbolos:", mask_nonnumeric.sum())

# Longitudes  
longitudes = df["TELEFONO"].dropna().str.replace(r"\D", "", regex=True).str.len()
print(longitudes.describe())


Con .0 al final: 3522
Con otros símbolos: 3556
count    6553.000000
mean        8.577598
std         0.782789
min         2.000000
25%         8.000000
50%         9.000000
75%         9.000000
max        24.000000
Name: TELEFONO, dtype: float64


In [316]:
import re

def limpiar_telefono(x):
    def clean_number(num_str):
        # 1) manejar notación científica
        try:
            if re.search(r"[eE]", num_str):
                num_str = str(int(float(num_str)))
        except:
            return "-1"
        # 2) quitar todo menos dígitos
        num_str = re.sub(r"\D+", "", num_str)
        if not num_str:
            return "-1"
        # 3) truncar si hay >8 dígitos
        if len(num_str) > 8:
            num_str = num_str[:8]
        # 4) invalidar si <7 dígitos
        if len(num_str) < 7:
            return "-1"
        # 5) asegurar 8 dígitos
        return num_str.zfill(8)

    if pd.isna(x):
        return pd.Series({
            "TELEFONO_LIMPIO": "-1",
            "TELEFONOS_ADICIONALES": ""
        })

    s = str(x).strip()
    # dividir en múltiplos separadores
    partes = re.split(r"[;\-/, ]+", s)

    # limpiar cada parte
    limpiados = [clean_number(p) for p in partes]
    # quedarnos sólo con los válidos
    validos = [n for n in limpiados if n != "-1"]

    if not validos:
        return pd.Series({
            "TELEFONO_LIMPIO": "-1",
            "TELEFONOS_ADICIONALES": ""
        })

    primary = validos[0]
    additional = validos[1:]

    return pd.Series({
        "TELEFONO_LIMPIO": primary,
        "TELEFONOS_ADICIONALES": ";".join(additional)
    })

In [317]:
# Aplicar la función y unir las dos columnas nuevas
df[["TELEFONO_LIMPIO", "TELEFONOS_ADICIONALES"]] = (
    df["TELEFONO"]
      .apply(limpiar_telefono)
)

In [318]:
# Ahora imprimimos las métricas sobre la columna limpia
print((df["TELEFONO_LIMPIO"] == "-1").sum(), "teléfonos inválidos")

51 teléfonos inválidos


In [319]:
print(df[['TELEFONO']].sample(5))

        TELEFONO
1488    66330312
718   22584390.0
1524    66334552
5907    47401633
4234  53897913.0


In [320]:
print(df["TELEFONOS_ADICIONALES"])

0        
1        
2        
3        
4        
       ..
6594     
6595     
6596     
6597     
6598     
Name: TELEFONOS_ADICIONALES, Length: 6599, dtype: object


In [321]:
long_post = df["TELEFONO_LIMPIO"]\
    .replace("-1", pd.NA)\
    .dropna()\
    .str.len()\
    .value_counts()
print("Distribución de longitudes tras limpieza:\n", long_post)

Distribución de longitudes tras limpieza:
 TELEFONO_LIMPIO
8    6548
Name: count, dtype: int64


### Razones de Limpieza para la Variable Teléfono

<small>

Decidimos primero normalizar todos los valores a cadenas de dígitos puros para garantizar la comparabilidad y evitar formatos mixtos (espacios, guiones, puntos o notación científica). Al eliminar cualquier carácter no numérico y convertir expresiones en “e” o con decimales a enteros, garantizamos que cada teléfono represente únicamente los dígitos esenciales. Truncar los números a ocho dígitos (o completar con ceros a la izquierda cuando falten) responde a la longitud estándar de los celulares en nuestro contexto, mientras que invalidar aquellas cadenas con menos de siete dígitos evita datos claramente erróneos o incompletos. Además, al extraer el primer número como “principal” y agrupar los adicionales en una columna separada, permitimos tanto el análisis prioritario de un contacto principal como la retención de otras vías de comunicación, sin duplicar registros.

</small>

## Limpieza de la variable "Director"

In [322]:
df = pd.read_csv("all_data.csv", dtype={"DIRECTOR": str})

In [323]:
def normalize_text(text: str) -> str:
    """
    Elimina tildes y acentos, pasa a mayúsculas y recorta espacios.
    """
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    return text.upper().strip()

In [324]:
INVALID_MARKERS = {"nan", "none", "", "---", "----", "-----", "x", "--"}

In [325]:
def limpiar_director(val: str) -> str:
    """
    1) Si es nulo o un marcador inválido → "SIN DIRECTOR"
    2) Si, tras strip(), len < 6 → "SIN DIRECTOR"
    3) En otro caso → normalizar tildes y pasar a mayúsculas
    """
    if pd.isna(val):
        return "SIN DIRECTOR"
    v = val.strip()
    if v.lower() in INVALID_MARKERS:
        return "SIN DIRECTOR"
    if len(v) < 6:
        return "SIN DIRECTOR"
    return normalize_text(v)

In [326]:
df["DIRECTOR_LIMPIO"] = df["DIRECTOR"].apply(limpiar_director)

In [327]:
print("Total filas:                       ", len(df))
print("Nulos originales:                  ", df["DIRECTOR"].isna().sum())
print("Reemplazados como 'SIN DIRECTOR': ", (df["DIRECTOR_LIMPIO"] == "SIN DIRECTOR").sum())
print("Valores únicos en limpio:          ", df["DIRECTOR_LIMPIO"].nunique())
print("Top 10 directores más frecuentes:\n", df["DIRECTOR_LIMPIO"].value_counts().head(10))

Total filas:                        6599
Nulos originales:                   26
Reemplazados como 'SIN DIRECTOR':  40
Valores únicos en limpio:           3820
Top 10 directores más frecuentes:
 DIRECTOR_LIMPIO
SIN DIRECTOR                        40
MARIA DOLORES PEREZ TUCHAN          10
HECTOR REYNALDO GOMEZ AGUILAR        9
SONIA JOSEFINA MORALES CAXAJ         8
MELVIN RAFAEL REYES LOPEZ            8
RONI ERNESTO RECINOS ESTRADA         8
FRANCISCO REVOLORIO LOPEZ            8
SILVIA MARLENI ALVARADO GUARDADO     8
SANDRA NOEMI ORTIZ ESTRADA           7
EDILSAR ROCAEL VAZQUEZ PEREZ         7
Name: count, dtype: int64


In [328]:
mapping = (
    df[["DIRECTOR", "DIRECTOR_LIMPIO"]]
      .dropna(subset=["DIRECTOR"])      
      .drop_duplicates()               
)

counts_raw_per_clean = (
    mapping
      .groupby("DIRECTOR_LIMPIO")["DIRECTOR"]
      .nunique()
)

variants = counts_raw_per_clean[counts_raw_per_clean > 1]

print("\nNombres normalizados que unifican >1 variante original:")
print(variants.sort_values(ascending=False).head(10))

print("\nEjemplos de variantes agrupadas:")
for clean_name, group in mapping.groupby("DIRECTOR_LIMPIO"):
    raws = list(group["DIRECTOR"])
    if len(raws) > 1:
        print(f"  {clean_name!r}  ←  {raws}")


Nombres normalizados que unifican >1 variante original:
DIRECTOR_LIMPIO
SIN DIRECTOR                              5
ANAHI DEL PILAR HERNANDEZ CHOCHOM         3
MARIA VERONICA AUYON OCAMPO DE GALICIA    3
BLANCA ROSA HERNANDEZ LOPEZ               2
ANGELICA DEL ROSARIO PATZAN QUISQUE       2
ANGEL FELIPE LARIOS MELENDEZ              2
CESAR FERNANDO ESCOBAR ZEPEDA             2
CARLA ELIZABETH MONTERROSO GOMEZ          2
CLAUDIA PATRICIA DAVILA RAMIREZ           2
DALIDA ANAYANZA HERNANDEZ RAMIREZ         2
Name: DIRECTOR, dtype: int64

Ejemplos de variantes agrupadas:
  'ANAHI DEL PILAR HERNANDEZ CHOCHOM'  ←  ['ANAHÍ DEL PILAR HERNÁNDEZ CHOCHOM', 'ANAHI DEL PILAR HERNÁNDEZ CHOCHÓM', 'ANAHÍ DEL PILAR HERNÁNDEZ CHOCHÓM']
  'ANGEL FELIPE LARIOS MELENDEZ'  ←  ['ANGEL FELIPE LARIOS MELENDEZ', 'ANGEL FELIPE LARIOS MELÉNDEZ']
  'ANGELICA DEL ROSARIO PATZAN QUISQUE'  ←  ['ANGÉLICA DEL ROSARIO PATZÁN QUISQUE', 'ANGELICA DEL ROSARIO PATZAN QUISQUE']
  'BLANCA ROSA HERNANDEZ LOPEZ'  ←  ['BLANCA 

### Razones de Limpieza para la Variable Director

<small>
Para la variable DIRECTOR reemplazamos los valores nulos y los marcadores inválidos (p. ej., nan, X, ---, cadenas vacías) por “SIN DIRECTOR”. La razón es asegurar que no existan valores faltantes o ruidosos que rompan agregaciones, filtros o uniones posteriores; además, “SIN DIRECTOR” comunica explícitamente la ausencia de dato y es más interpretable que NaN o símbolos. También marcamos como inválidos los nombres demasiado cortos (menos de 6 caracteres), porque en tu análisis aparecían entradas como "X" o "---" que son errores de digitación o registros incompletos; esto reduce outliers y mejora la calidad general.

Luego normalizamos todos los nombres eliminando tildes y diacríticos, y llevándolos a mayúsculas. El objetivo es evitar duplicidades por acentos (p. ej., “MARÍA” vs “MARIA”) y variaciones mínimas de formato que inflan el número de únicos y complican cualquier conteo, deduplicación o emparejamiento. Este paso te permitió unificar variantes (con/sin tilde) bajo un mismo rótulo, haciendo que métricas como “top de directores” o cualquier agrupación por nombre sean consistentes y comparables.
</small>

## Limpieza de la variable "Dirección"

In [329]:
df = pd.read_csv("all_data.csv", dtype={"DIRECCION": str})

In [330]:
def normalize_text(text: str) -> str:
    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = re.sub(r"\s+", " ", text).strip().upper()
    return text

In [331]:
INVALID_MARKERS = {"", "nan", ".", '""'}
ABBR_MAP = {
    "KM":  "KILOMETRO",
    "NO":  "NUMERO",
    "AV":  "AVENIDA",
    "AVE": "AVENIDA",
    "BO":  "BARRIO",
}

In [332]:
abbr_pattern = re.compile(r'\b([A-ZÁÉÍÓÚÑ]{2,5})\.\b')

In [333]:
def limpiar_direccion(val: str) -> str:
    if pd.isna(val):
        return "SIN DIRECCION"
    v = val.strip()
    if v.lower() in INVALID_MARKERS:
        return "SIN DIRECCION"

    # Eliminar comillas
    v = v.replace('"', '')
    # Expandir abreviaturas (KM., NO., AV., AVE., BO.)
    v = abbr_pattern.sub(lambda m: ABBR_MAP.get(m.group(1), m.group(1)), v)
    # Eliminar ordinales ª/º
    v = re.sub(r'(\d+)[ºª]\b', r'\1', v)
    # Eliminar 1RA, 2DA, etc.
    v = re.sub(r'\b(\d+)(?:RA|DA|RO|TA)[\.,]?\b', r'\1', v, flags=re.IGNORECASE)
    # Quitar puntos y comas sobrantes
    v = v.replace('.', '').replace(',', '')

    return normalize_text(v)

In [334]:
df["DIRECCION_LIMPIA"] = df["DIRECCION"].apply(limpiar_direccion)

In [335]:
print("Total filas:                  ", len(df))
print("Reemplazados como 'SIN DIRECCION':", (df["DIRECCION_LIMPIA"]=="SIN DIRECCION").sum())
print("Valores únicos limpios:       ", df["DIRECCION_LIMPIA"].nunique())
print("Ejemplos top-10 direcciones:\n", df["DIRECCION_LIMPIA"].value_counts().head(10))

Total filas:                   6599
Reemplazados como 'SIN DIRECCION': 4
Valores únicos limpios:        4103
Ejemplos top-10 direcciones:
 DIRECCION_LIMPIA
CABECERA MUNICIPAL           199
BARRIO EL CENTRO              51
BARRIO SAN SEBASTIAN          16
BARRIO EL CALVARIO            14
ZONA 1 PLAYA GRANDE           13
ALDEA PLAYITAS                12
ALDEA CAMOJALLITO             11
ALDEA FRONTERAS RIO DULCE     11
CALLE PRINCIPAL               11
CANTON SAN SEBASTIAN          10
Name: count, dtype: int64


In [336]:
mapping = (
    df[["DIRECCION", "DIRECCION_LIMPIA"]]
      .dropna(subset=["DIRECCION"])
      .drop_duplicates()
)
counts_raw_per_clean = mapping.groupby("DIRECCION_LIMPIA")["DIRECCION"].nunique()
variants = counts_raw_per_clean[counts_raw_per_clean > 1]
print("\nDirecciones normalizadas que unifican >1 variante original:")
print(variants.head(10))

for clean, group in mapping.groupby("DIRECCION_LIMPIA"):
    raws = list(group["DIRECCION"])
    if len(raws) > 1:
        print(f"\n{clean!r}  ← variantes:\n  {raws}")


Direcciones normalizadas que unifican >1 variante original:
DIRECCION_LIMPIA
0 AVENIDA 0-06 CANTON AGUA TIBIA                                               2
0 AVENIDA 1A CALLE NO 002 ZONA 4 BARRIO SANTO DOMINGO                          2
0 AVENIDA A 4-50 COLONIA LOS ALAMOS ZONA 6                                     2
0 CALLE 0-65 ZONA 1 CALZADA VIRGILIO RODRIGUEZ MACAL SANTA ELENA DE LA CRUZ    2
0 CALLE 1A AVENIDA LOTE 6 COLONIA VINICIO CEREZO AREVALO                       2
0 CALLE 4-01 ZONA 4                                                            2
0 CALLE A CANTON XE KAQASIIWAAN ZONA 1                                         2
0 CALLE Y 36 AVENIDA ZONA 8 BARRIO GARIBALDI                                   2
1 AVENIDA 1-41 ZONA 1                                                          2
1 CALLE LOTE 650 BARRIO LA ARENERA                                             2
Name: DIRECCION, dtype: int64

'0 AVENIDA 0-06 CANTON AGUA TIBIA'  ← variantes:
  ['0 AVENIDA 0-06, CANTON AGUA 

In [337]:
patterns = {
    "abreviaciones": re.compile(r'\b(?:KM|NO|AV|AVE|BO)\.'),
    "ordinales_ªº":    re.compile(r'\d+[ºª]\b'),
    "ordinales_RA":    re.compile(r'\b\d+(?:RA|DA|RO|TA)\.?\b', re.IGNORECASE),
}

print("Antes de limpiar")
for name, pat in patterns.items():
    cnt = df["DIRECCION"].str.contains(pat, na=False).sum()
    print(f"{name:15s}: {cnt} ocurrencias")

print("\nDespués de limpiar")
for name, pat in patterns.items():
    cnt = df["DIRECCION_LIMPIA"].str.contains(pat, na=False).sum()
    print(f"{name:15s}: {cnt} ocurrencias")


Antes de limpiar
abreviaciones  : 533 ocurrencias
ordinales_ªº   : 24 ocurrencias
ordinales_RA   : 481 ocurrencias

Después de limpiar
abreviaciones  : 0 ocurrencias
ordinales_ªº   : 0 ocurrencias
ordinales_RA   : 0 ocurrencias


### Razones de Limpieza para la Variable Dirección

<small>
Para DIRECCION, primero tratamos valores vacíos o irrelevantes (como "." o NaN) estableciéndolos en “SIN DIRECCION”. Esto homogeneiza los faltantes con un valor explícito que no confunde los análisis ni crea categorías artificiales. Luego eliminamos comillas dobles que envolvían algunas direcciones, ya que solo introducían ruido sin valor semántico. También expandimos abreviaciones frecuentes (p. ej., KM. → “KILOMETRO”, NO. → “NUMERO”, AV./AVE. → “AVENIDA”, BO. → “BARRIO”), porque estas abreviaturas aparecían con baja consistencia y su expansión mejora la legibilidad y la uniformidad para búsquedas y emparejamientos.

Adicionalmente, removimos los indicadores ordinales tanto con diacríticos (1º, 2ª) como escritos (1RA, 2DA, 3RO, 4TA, con o sin punto), para normalizar patrones como “1RA.AVE.” → “1 AVENIDA”. Esto evita que los mismos tramos de dirección aparezcan con variantes formales que fragmentan los conteos. Por último, eliminamos tildes/diacríticos y pasamos a mayúsculas, y normalizamos separadores (p. ej., guiones con espaciado), logrando que cadenas equivalentes se reconozcan como iguales. Estos cambios reducen duplicidades, mejoran la calidad del texto y facilitan reportes, filtros y uniones confiables sin perder el significado de la dirección.
</small>

## Limpieza de la variable "Supervisor"

In [3]:
df = pd.read_csv("all_data.csv", dtype={"SUPERVISOR": str})

INVALID_MARKERS = {"", "nan", "none", "---", "----", "-----", "x", "--"}
TITULOS = {
    "LIC", "LIC.", "ING", "ING.", "PROF", "PROF.", "DRA", "DRA.", "DR", "DR.",
    "MSC", "M.SC", "MS", "M.A", "M.A.", "MA", "ARQ", "ARQ.", "MTRO", "MTRO.", "MTTRA", "MTTRA."
}

In [4]:
def _quitar_titulos(s: str) -> str:
    tokens = s.replace(",", " ").split()
    tokens = [t for t in tokens if t.upper().strip(".") not in {t.strip(".") for t in TITULOS}]
    return " ".join(tokens)

def _normalize_text(text: str) -> str:
    # Elimina tildes/diacríticos, pasa a mayúsculas y colapsa espacios
    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = text.upper()
    # Sustituye signos y puntuación por espacio; deja solo letras, Ñ y espacios
    text = re.sub(r"[\"'.,;/\\\-]", " ", text)
    text = re.sub(r"[^A-ZÑ ]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def limpiar_supervisor(val: str) -> str:
    """
    Reglas:
    1) Nulos/Marcadores inválidos -> "SUPERVISOR NO ASIGNADO"
    2) Remover títulos (LIC., ING., DR., etc.)
    3) Normalizar (mayúsculas, sin tildes, sin símbolos, espacios simples)
    4) Validar al menos 2 palabras y longitud razonable
    """
    if pd.isna(val):
        return "SUPERVISOR NO ASIGNADO"
    raw = str(val).strip()
    if raw.lower() in INVALID_MARKERS:
        return "SUPERVISOR NO ASIGNADO"

    raw = _quitar_titulos(raw)
    clean = _normalize_text(raw)

    # Validación mínima
    if len(clean) < 6 or len(clean.split()) < 2:
        return "SUPERVISOR NO ASIGNADO"

    return clean

In [5]:
df["SUPERVISOR_LIMPIO"] = df["SUPERVISOR"].apply(limpiar_supervisor)

In [6]:
print("Total filas:                        ", len(df))
print("Nulos originales:                   ", df["SUPERVISOR"].isna().sum())
print("Asignados como 'SUPERVISOR NO ASIGNADO':",
      (df["SUPERVISOR_LIMPIO"] == "SUPERVISOR NO ASIGNADO").sum())
print("Valores únicos (original):          ", df["SUPERVISOR"].nunique())
print("Valores únicos (limpio):            ", df["SUPERVISOR_LIMPIO"].nunique())
print("\nTop 10 supervisores (limpio):\n", df["SUPERVISOR_LIMPIO"].value_counts().head(10))

Total filas:                         6599
Nulos originales:                    0
Asignados como 'SUPERVISOR NO ASIGNADO': 0
Valores únicos (original):           599
Valores únicos (limpio):             591

Top 10 supervisores (limpio):
 SUPERVISOR_LIMPIO
MIGUEL ANGEL ARMAS ROCHA                          190
CARLOS HUMBERTO GONZALEZ DE LEON                  170
JUAN ENRIQUE MARTINEZ SOLANO                      106
REMY ARTURO SINAY GUDIEL                           94
ELSA YADIRA MARTINEZ PEREZ                         84
IDALIA DEL ROSARIO LOPEZ SANDOVAL DE PAIZ          81
MILTON ALONSO ALVAREZ FUENTES                      81
ELENA ELIZABETH SUCHITE GARNICA DE QUINTANILLA     78
LUDVIN RICARDO URRUTIA LORENTI                     77
JUAN FRANCISCO GODOY DAVILA                        74
Name: count, dtype: int64


In [7]:
mapping_sup = (
    df[["SUPERVISOR", "SUPERVISOR_LIMPIO"]]
      .dropna(subset=["SUPERVISOR"])
      .drop_duplicates()
)

counts_raw_per_clean = (
    mapping_sup
      .groupby("SUPERVISOR_LIMPIO")["SUPERVISOR"]
      .nunique()
      .sort_values(ascending=False)
)

In [8]:
print("\nNombres normalizados que unifican >1 variante original (top 10):")
print(counts_raw_per_clean[counts_raw_per_clean > 1].head(10))

ejemplos = 0
for clean_name, group in mapping_sup.groupby("SUPERVISOR_LIMPIO"):
    raws = list(group["SUPERVISOR"])
    if len(raws) > 1:
        print(f"\n'{clean_name}'  ← variantes:\n  {raws[:6]}")
        ejemplos += 1
    if ejemplos >= 10:
        break


Nombres normalizados que unifican >1 variante original (top 10):
SUPERVISOR_LIMPIO
MIGUEL AJPOP VASQUEZ                2
JUAN CARLOS MORALES GONZALEZ        2
CARLOS HUMBERTO GONZALEZ DE LEON    2
JULIO RENE NOTZ CUTZAL              2
JOSE DE JESUS CAMPOSECO MENDOZA     2
JUAN JOSE TOBAR TEBALAN             2
AUGUSTO MANUEL ANDRES SEBASTIAN     2
FIDEL OMAR HERNANDEZ RAMIREZ        2
Name: SUPERVISOR, dtype: int64

'AUGUSTO MANUEL ANDRES SEBASTIAN'  ← variantes:
  ['AUGUSTO MANUEL ANDRÉS SEBASTIÁN', 'AUGUSTO MANUEL ANDRES SEBASTIAN']

'CARLOS HUMBERTO GONZALEZ DE LEON'  ← variantes:
  ['CARLOS HUMBERTO GONZÁLEZ DE LEÓN', 'CARLOS HUMBERTO GONZALEZ DE LEON']

'FIDEL OMAR HERNANDEZ RAMIREZ'  ← variantes:
  ['FIDEL OMAR HERNÁNDEZ RAMÍREZ', 'FIDEL OMAR HERNANDEZ RAMIREZ']

'JOSE DE JESUS CAMPOSECO MENDOZA'  ← variantes:
  ['JOSÉ DE JESÚS CAMPOSECO MÉNDOZA', 'JOSÉ DE JESÚS CAMPOSECO MENDOZA']

'JUAN CARLOS MORALES GONZALEZ'  ← variantes:
  ['JUAN CARLOS MORALES GONZALEZ', 'JUAN CARLOS MORAL

### Razones de Limpieza para la Variable Supervisor

Se remueven títulos y prefijos (p. ej., **LIC.**, **ING.**, **DR.**) y se normaliza a mayúsculas sin tildes para unificar variantes ortográficas y de formato. Se eliminan símbolos y se colapsan espacios para evitar duplicidad por puntuación. Se marcan como **"SUPERVISOR NO ASIGNADO"** los nulos, marcadores inválidos o entradas que no alcanzan un mínimo de calidad (menos de 2 palabras o longitud muy corta).